In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import tempfile
import os

d:\Anaconda\envs\POC_STT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)

cuda


In [5]:
token_model ='skt/kogpt2-base-v2'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(token_model)
model = AutoModelForCausalLM.from_pretrained(token_model).to(device)

In [7]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [8]:
#프롬포트 기반 LLM 생성기(skt/kogpt2용)
def generate_kogpt(prompt: str, max_length=512):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    input_ids = inputs['input_ids']
    print(f"[DEBUG] Prompt token length: {input_ids.shape[1]}")
    print(f"[DEBUG] Max token id: {input_ids.max().item()}, Vocab size: {tokenizer.vocab_size}")
    inputs = inputs.to(device)
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.8)
    prompt_len = input_ids.shape[1]
    generated_tokens = outputs[0][prompt_len:]
    generated_only = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return generated_only


#프롬포트 기반 LLM 생성기(EleutherAI/polyglot-ko-1.3B)
# def generate_kogpt(prompt):
#     inputs = tokenizer(prompt, return_tensors='pt')
    
#     # ✅ token_type_ids 제거
#     if 'token_type_ids' in inputs:
#         del inputs['token_type_ids']
    
#     print(f"[DEBUG] Prompt token length: {inputs['input_ids'].shape[1]}")
    
#     inputs = inputs.to(device)
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=256,
#         do_sample=True,
#         temperature=0.8
#     )
#     generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     generated_only = generated[len(prompt):].strip()
#     return generated_only


# #디버깅용
# def generate_kogpt(prompt: str, max_length=512):
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
#     input_ids = inputs['input_ids'][0]
#     print(f"Input max token id: {input_ids.max()}, vocab size: {tokenizer.vocab_size}")
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.8)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
# 단순한 토큰 단위 분할
def chunk_text(text, max_chars=600):
    chunks = []
    for i in range(0, len(text), max_chars):
        chunks.append(text[i:i+max_chars])
    return chunks


In [10]:
def convert_to_written(spoken_text):
    prompt = f"""
아래 구술체를 자연스럽고 정중한 문어체로 변환해 주세요.

예시)
구술체: "오늘 날씨 진짜 좋았어."
문어체: "오늘 날씨가 매우 좋았습니다."

[변환할 텍스트]
{spoken_text}
"""
    return generate_kogpt(prompt)


In [11]:
#요약
def summarize(text):
    prompt = f"""다음 내용을 요약해 주세요 (중요한 장소, 인물, 경험 중심으로 간결하게):\n{text}\n\n요약:"""
    return generate_kogpt(prompt)

In [12]:
#동화 생성
def generate_fairytale(summary):
    prompt = f"""다음 내용을 바탕으로 초등학생용 창의적인 동화를 만들어주세요. 제목도 포함해주세요.\n\n내용:\n{summary}\n\n동화:"""
    return generate_kogpt(prompt, max_length=768)

In [13]:
text_file = f'D:/gap_year/poc-story-field-ai/STORY/LLM/travel_test.txt'
with open(text_file, 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

여행, 여행하면 혹시 기억에 남는 여행이라든지 아니면 엄청 좋았던 여행에 대해서 말씀해 주실까 있을까요? 네. 여행하면 배를 타고, 인차를 세 배를 타고 중국 단동으로 가서 그 에이에 백두산. 음. 백두산 간 적이
 있는데요. 네. 우리는 이제 문화원에서 가는 거예요. 그래서 약 한 120명 정도? 음. 많이가 좋네요. 그런데 그 문화원에 120명 정도 백두산만 간 게 아니라 러시아를 10박 11일, 인도 10박 11일, 뭐 중국, 뭐 할 것  
없이 이번에는 그 울릉도 갔다 왔고요. 그래서 늘 그 단체로 가기 때문에 문화원, 우리 왓프의 그 문화원에서 가기 때문에 굉장히 그 숙소 뿐만 아니라 여행지, 아니면 러시아를 갔다는 러시아에서 그 해산사를 딱 투 
어요. 러시아 대학의, 러시아 대학원의 한국 학생이 러시아 공부를 하는 사람을 해설하는 거예요. 음. 그 너무 재밌고, 볼만하고. 거기서 어떤 거냐면 우리가 그 시베리아 그 뭐죠? 그 열차 있죠? 횡단 열차. 횡단 열 
차에 타고, 여름이 어디에서 이렇게 축하해서 어디까지 딱 가요. 거기 내려가지고, 하루 먹고 시컨 거기서 이제 놀고 또 다시 어디를 이동한다면 거기서 다시 또 시베리아에 타고 또 어디까지 또 가는 거예요. 음. 거 
기서 그 시빡 시비를 몽골까지 갔거든요. 음. 그래서 거기 가던 그 여행, 여행지. 그러니까 굉장히 그 시베리아 열차, 횡단 열차가 굉장히 거기 열악해요. 음. 시단시도 열악하고. 음. 거기 2층 침대, 거기 2층 침대를
 막 4명이서 거기 올라가서 굉장히 불편하지만. 음. 그리고 굉장히 시끄럽고. 음. 그래서 밖을 보면 그냥 막막 되게 깜깜하잖아요. 밤에만 이동하니까. 그렇죠. 어디가 어딘지 잘 몰라요. 음. 그래서 아침에 내려서 다
시 이동해서. 음. 그 무거운 친구가 이렇게 막 내려서 어저어로 가서 또 하고 또 후대로 가서 진동하고 또 다시 여행 가면 거기 이제 그 회사자가 고고마다 다르잖아요. 네. 그 회사에 드는 곳도 되게 좋았고. 음. 그 
여행은 이제 또 다 같은 문

In [14]:
chunks = chunk_text(content, max_chars=600)  # 문자 기준 chunking

written_parts = [convert_to_written(chunk) for chunk in chunks]
written = '\n\n'.join(written_parts)
print(written)

[DEBUG] Prompt token length: 304
[DEBUG] Max token id: 50082, Vocab size: 51200


d:\Anaconda\envs\POC_STT\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[DEBUG] Prompt token length: 343
[DEBUG] Max token id: 50082, Vocab size: 51200
[DEBUG] Prompt token length: 332
[DEBUG] Max token id: 50082, Vocab size: 51200
[DEBUG] Prompt token length: 263
[DEBUG] Max token id: 51158, Vocab size: 51200
그때 그 문화원에서 러시아 가자고 한 거예요. 저는 러시아 대학원에 갔습니다. 러시아 대학교에 갔지만. 우리 왓프의 그 호텔에서 그 호텔을 간다고 해서 그 호텔에 가서 하룻밤 묵고. 그런데 그 시간에 그 러시아 호텔에서 제가 러시아 문화원에서 갔습니다 해서 우리는 그 호텔에서 하룻밤 묵고 가서 이제 막 그 러시아 문화원에서 러시아 대학원이기 때문에 거기서 또 하룻밤에 잤습니다. 그 호텔에서 그 러시아 문화원 있는 줄 알았어요? 제가 아까 그 문화원에서 그 러시아 대학교에 갔는데. 그 러시아 대학교는 한국 대학원이기도 하고. 그 러시아 대학교에서 한국 대학원이기 때문에 그 러시아 대학교에도 한국 대학원이 있었습니다. 그 교수님이 바로 그 교수님이었어요. 그 러시아 대학교는 한국 대학원이기도 해서 그 교수님이 이제 러시아에서 하는 그 강의 들으면서 그런 강의를 하면서 그 교수님 강의를 듣고 또 그 교수님이 한국 대학원에 계셨는데. 그 한국 대학교에 가서 그 교수님 강의를 들으면서 한국 대학에서 그런 강의 들으면서 한국 대학원에 가는데 그 교수님이 강의 들으면서 그 교수님 강의를 듣고 또 교수가 한국 대학에서 강의 들으면서 교수님 강연 들으면서 한국 대학에서 강의 듣고 하는 그런 강의를 들으면서 한국 대학에서 강의 들으면서 한국 대학에서 강의 들으면서 교수님

그렇게 이제 가고 있는데. 이제 우리가 가는 곳은 가기로 하고 왔어요. 그래가지고. 어.
너무 좋았어. 진짜 너무 좋았어. 음. 그리고 또 다른 친구들. 음. 그리고 또 롯데가랑도 가고 